In [5]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# path to the database - change it if needed
path = 'D:\\Python\\MachineLearning\\MachineLearning\\week05\\data\\face_data\\face_data\\'

ids = range(1, 16) # 15 people
states = ['centerlight', 'glasses', 'happy', 'leftlight','noglasses', 'normal', 'rightlight','sad','sleepy', 'surprised', 'wink' ]
prefix = 'subject'
surfix = '.png' #file extension is png

# open one picture to get the image's size
fn = prefix + '01.' + states[0] + surfix
im = cv2.imread(path + fn, 0)

h = im.shape[0] # hight
w = im.shape[1] # width

D = h * w
N = len(states)*15
print(N, D, h, w)

X = np.zeros((D, N))

# collect all data
count = 0
# there are 15 people
for person_id in range(1, 16):
    for state in states:
        # get name of each image file
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + surfix
        # open the file and read as grey image
        tmp = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)
        # then add image to dataset X
        X[:, count] = tmp.reshape(D)
        count += 1
        
# Tạo mảng y chứa nhãn cho mỗi mẫu dữ liệu
# Khởi tạo mảng y
y = np.zeros(165)
# Chuyển y thành mảng numpy để sử dụng trong train_test_split
y = states * 15
y = np.array(y)

165 77760 243 320


Áp dụng cho dữ liệu ảnh khuôn mặt (của 15 người, mỗi người có 11 trạng thái):

a. Giảm số chiều dữ liệu xuống còn 500.

In [2]:
from sklearn.decomposition import PCA

# Initialize PCA with n_components = min(n_samples, n_features)
pca = PCA(50)

# Fit PCA to the data X
X_pca = pca.fit_transform(X.T)  # Transpose X to fit PCA (samples as rows, features as columns)

print("Original shape:", X.shape)
print("Shape after PCA:", X_pca.shape)


Original shape: (77760, 165)
Shape after PCA: (165, 50)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Multinomial Logistic Regression
logistic_regression = LogisticRegression(multi_class='multinomial', solver='sag', max_iter=5000)
logistic_regression.fit(X_train, y_train)
y_pred_lr = logistic_regression.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("Accuracy of Multinomial Logistic Regression:", accuracy_lr)

# Naïve Bayes
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)
y_pred_nb = naive_bayes.predict(X_test)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print("Accuracy of Naïve Bayes:", accuracy_nb)


Accuracy of Multinomial Logistic Regression: 0.16
Accuracy of Naïve Bayes: 0.1


In [4]:
import numpy as np
import cv2
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# Load and preprocess the new images
new_image_paths = ['D:\\Python\\MachineLearning\\MachineLearning\\week05\\data\\face_data\\face_data\\subject01.centerlight.png',
                   'D:\\Python\\MachineLearning\\MachineLearning\\week05\\data\\face_data\\face_data\\subject01.glasses.png',
                   'D:\\Python\\MachineLearning\\MachineLearning\\week05\\data\\face_data\\face_data\\subject01.happy.png',
                   'D:\\Python\\MachineLearning\\MachineLearning\\week05\\data\\face_data\\face_data\\subject01.leftlight.png',
                   'D:\\Python\\MachineLearning\\MachineLearning\\week05\\data\\face_data\\face_data\\subject01.normal.png']
new_images = []
for path in new_image_paths:
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # Resize image to match the dimensions of the original dataset
    img_resized = cv2.resize(img, (243, 320))
    new_images.append(img_resized.flatten())  # Flatten image to match the shape of data for PCA

# Convert new images to numpy array
X_new = np.array(new_images)

# Transform new images using the PCA model trained on the original dataset
X_new_pca = pca.transform(X_new)

# Use the trained model to predict labels for the new images
y_pred_lr_new = logistic_regression.predict(X_new_pca)
y_pred_nb_new = naive_bayes.predict(X_new_pca)

# Print predicted labels for the new images
print("Predicted labels using Multinomial Logistic Regression:", y_pred_lr_new)
print("Predicted labels using Naïve Bayes:", y_pred_nb_new)


Predicted labels using Multinomial Logistic Regression: ['noglasses' 'noglasses' 'noglasses' 'leftlight' 'noglasses']
Predicted labels using Naïve Bayes: ['sleepy' 'normal' 'normal' 'rightlight' 'normal']
